In [ ]:
import cv2  
import numpy as np  


In [ ]:
camera = cv2.VideoCapture('./3.mp4')  
bs = cv2.createBackgroundSubtractorKNN(detectShadows = True)

In [ ]:

while(camera.isOpened()):  
    ret, frame = camera.read()  
    cv2.imshow('image', frame)  
    k = cv2.waitKey(100)  
    #q键退出
    if (k & 0xff == ord('q')):  
        break  

camera.release()  
cv2.destroyAllWindows()

In [ ]:
import cv2  
import numpy as np
import datetime
camera = cv2.VideoCapture('./3.mp4')  
bs = cv2.createBackgroundSubtractorKNN(detectShadows = True)# 背景分割器，该函数计算了前景掩码
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
Gaussian_kernel_size = (5, 5);
Gaussian_sigma = 1.5;
timeF = 1
time_i=1
if camera.isOpened(): #判断是否正常打开
    ret, frame0 = camera.read()
else:
    ret = False
while ret:   #循环读取视频帧
    ret, frame0 = camera.read()
    if(time_i%timeF == 0): #每隔timeF帧进行存储操作
        #print(frame0.shape[0])   #0,h;1,w;2,c
        frame = cv2.GaussianBlur(frame0, Gaussian_kernel_size, Gaussian_sigma);
        fgmask = bs.apply(frame)  
        fg2 = fgmask.copy()  
        # 二值化阈值处理，前景掩码含有前景的白色值以及阴影的灰色值，在阈值化图像中，将非纯白色（244~255）的所有像素都设为0，而不是255
        th = cv2.threshold(fg2,200,255,cv2.THRESH_BINARY)[1]  
        # 形态学膨胀
        dilated = cv2.dilate(th,es,iterations = 2)  
        # 该函数计算一幅图像中目标的轮廓
        image, contours, hier = cv2.findContours(dilated,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    
        for c in contours:  
            if cv2.contourArea(c) > 10000:  
                (t_x,t_y,t_w,t_h) = cv2.boundingRect(c)
            
                if(150<t_w<600 and 150<t_h<600 and (frame0.shape[0]/2)<(t_y+t_h)<(frame0.shape[0]/2+100)):
                    cv2.rectangle(frame0,(t_x,t_y),(t_x+t_w,t_y+t_h),(255,255,0),2) 
                    now_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
                    print(now_time)
                    print('target location is',(t_x,t_y,t_w,t_h))
                    roi_img = frame0[t_y:(t_y+t_h), t_x:(t_x+t_w) ,:]
                    filename_roi = now_time + '.jpg'
                    cv2.imwrite(filename_roi,roi_img)
                    

              
        #cv2.imshow("mog",fgmask)  
        #cv2.imshow("thresh",th)  
        cv2.imshow("detection",frame0)
    time_i = time_i + 1
    if(time_i>=300):
        time_i = 0
    if cv2.waitKey(20) & 0xff == 27:  
        break       
    
camera.release()  
cv2.destroyAllWindows()  

import cv2  
import numpy as np
import datetime
camera = cv2.VideoCapture('./3.mp4')  
bs = cv2.createBackgroundSubtractorKNN(detectShadows = True)# 背景分割器，该函数计算了前景掩码
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
Gaussian_kernel_size = (5, 5);
Gaussian_sigma = 1.5;

if camera.isOpened(): #判断是否正常打开
    ret, frame0 = camera.read()
else:
    ret = False

with open('InfoDetect.txt', 'w') as file_detect:
    while ret:   #循环读取视频帧
        ret, frame0 = camera.read()
        #灰度化处理
        #framegrey=cv2.cvtColor(frame0,cv2.COLOR_BGR2GRAY)
        #gray_lap = cv2.Laplacian(framegrey,cv2.CV_16S,ksize = 3)
        #dst = cv2.convertScaleAbs(gray_lap)
    
        #if (qingxidu>80): #每隔timeF帧进行存储操作
            #print(frame0.shape[0])   #0,h;1,w;2,c
        frame = cv2.GaussianBlur(frame0, Gaussian_kernel_size, Gaussian_sigma);
        fgmask = bs.apply(frame)  
        fg2 = fgmask.copy()  
        # 二值化阈值处理，前景掩码含有前景的白色值以及阴影的灰色值，在阈值化图像中，将非纯白色（244~255）的所有像素都设为0，而不是255
        th = cv2.threshold(fg2,200,255,cv2.THRESH_BINARY)[1]  
        # 形态学膨胀
        dilated = cv2.dilate(th,es,iterations = 2)  
        # 该函数计算一幅图像中目标的轮廓
        image, contours, hier = cv2.findContours(dilated,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    
        for c in contours:  
            if cv2.contourArea(c) > 10000:  
                (t_x,t_y,t_w,t_h) = cv2.boundingRect(c)
            
                if(200<t_w<600 and 200<t_h<600 and (3*frame0.shape[0]/4)<(t_y+t_h)):
                    now_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
                    print(now_time)
                    print('target location is',(t_x,t_y,t_w,t_h))
                    roi_img = frame0[t_y:(t_y+t_h), t_x:(t_x+t_w) ,:]
                    framegrey=cv2.cvtColor(roi_img,cv2.COLOR_BGR2GRAY)
                    cv2.imshow("roi_img",roi_img)
                    qingxidu = cv2.Laplacian(framegrey,cv2.CV_64F).var()
                    cv2.rectangle(frame0,(t_x,t_y),(t_x+t_w,t_y+t_h),(255,255,0),2) 
                    contralIn = input('请输入d检测或者p放弃：')
                    if(qingxidu>180 and contralIn=='d'):
                        file_detect.write('时间：'+now_time + '\n')
                        file_detect.write('车牌号：'+ '\n')
                        file_detect.write('地点：三环路蓝天立交段由东向西'+ '\n')
                        file_detect.write('车辆位置：top_x=%d, top_y=%d, width=%d, height=%d \n'%(t_x,t_y,t_w,t_h))
                        file_detect.write('车辆颜色：'+ '\n')
                        file_detect.write('车辆型号：'+ '\n')
                        file_detect.write('车辆图片路径：./' + now_time + '.jpg'+ '\n')
                        file_detect.write('车辆视频路径：./3.mp4'+ '\n')
                        
                        filename_roi = now_time + '.jpg'
                        cv2.imwrite(filename_roi,roi_img)
              
            #cv2.imshow("mog",fgmask)  
            #cv2.imshow("thresh",th)  
        cv2.imshow("detection",frame0)

        if cv2.waitKey(33) & 0xff == 27:  
            break       
camera.release()  
cv2.destroyAllWindows()  

In [ ]:
import cv2
import numpy as np import datetime camera = cv2.VideoCapture('./3.mp4')
bs = cv2.createBackgroundSubtractorKNN(detectShadows = True)# 背景分割器，该函数计算了前景掩码 es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) Gaussian_kernel_size = (5, 5); Gaussian_sigma = 1.5;

if camera.isOpened(): #判断是否正常打开 ret, frame0 = camera.read() else: ret = False

with open('InfoDetect.txt', 'w') as file_detect: while ret: #循环读取视频帧 ret, frame0 = camera.read()

    #灰度化处理
    #framegrey=cv2.cvtColor(frame0,cv2.COLOR_BGR2GRAY)
    #gray_lap = cv2.Laplacian(framegrey,cv2.CV_16S,ksize = 3)
    #dst = cv2.convertScaleAbs(gray_lap)

    #if (qingxidu>80): #每隔timeF帧进行存储操作
        #print(frame0.shape[0])   #0,h;1,w;2,c
    frame = cv2.GaussianBlur(frame0, Gaussian_kernel_size, Gaussian_sigma);
    fgmask = bs.apply(frame)  
    fg2 = fgmask.copy()  
    # 二值化阈值处理，前景掩码含有前景的白色值以及阴影的灰色值，在阈值化图像中，将非纯白色（244~255）的所有像素都设为0，而不是255
    th = cv2.threshold(fg2,200,255,cv2.THRESH_BINARY)[1]  
    # 形态学膨胀
    dilated = cv2.dilate(th,es,iterations = 2)  
    # 该函数计算一幅图像中目标的轮廓
    image, contours, hier = cv2.findContours(dilated,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  

    for c in contours:  
        if cv2.contourArea(c) > 10000:  
            (t_x,t_y,t_w,t_h) = cv2.boundingRect(c)

            if(200<t_w<600 and 200<t_h<600 and (3*frame0.shape[0]/4)<(t_y+t_h)):
                now_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
                print(now_time)
                print('target location is',(t_x,t_y,t_w,t_h))
                roi_img = frame0[t_y:(t_y+t_h), t_x:(t_x+t_w) ,:]
                framegrey=cv2.cvtColor(roi_img,cv2.COLOR_BGR2GRAY)
                cv2.imshow("roi_img",roi_img)
                qingxidu = cv2.Laplacian(framegrey,cv2.CV_64F).var()
                cv2.rectangle(frame0,(t_x,t_y),(t_x+t_w,t_y+t_h),(255,255,0),2) 
                contralIn = input('请输入d检测或者p放弃：')
                if(qingxidu>180 and contralIn=='d'):
                    file_detect.write('时间：'+now_time + '\n')
                    file_detect.write('车牌号：'+ '\n')
                    file_detect.write('地点：三环路蓝天立交段由东向西'+ '\n')
                    file_detect.write('车辆位置：top_x=%d, top_y=%d, width=%d, height=%d \n'%(t_x,t_y,t_w,t_h))
                    file_detect.write('车辆颜色：'+ '\n')
                    file_detect.write('车辆型号：'+ '\n')
                    file_detect.write('车辆图片路径：./' + now_time + '.jpg'+ '\n')
                    file_detect.write('车辆视频路径：./3.mp4'+ '\n')

                    filename_roi = now_time + '.jpg'
                    cv2.imwrite(filename_roi,roi_img)

        #cv2.imshow("mog",fgmask)  
        #cv2.imshow("thresh",th)  
    cv2.imshow("detection",frame0)

    if cv2.waitKey(33) & 0xff == 27:  
        break       

camera.release()
cv2.destroyAllWindows()